In [1]:
## Connecting to the database
import mysql.connector as mysql
%store -r restaurant_dict
%store -r cuisines_dict

## importing 'mysql.connector' as mysql for convenient

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
db = mysql.connect(
    host = "127.0.0.1",
    user = "root",
    passwd = "root"
)

print(db) # it will print a connection object if everything is fine

In [2]:
## creating an instance of 'cursor' class which is used to execute the 'SQL' statements in 'Python'
cursor = db.cursor()

## creating a databse called 'datacamp'
## 'execute()' method is used to compile a 'SQL' statement
## below statement is used to create tha 'datacamp' database
cursor.execute("CREATE DATABASE IF NOT EXISTS restaurantdata")
cursor.execute("USE restaurantdata")
print(cursor.execute("SHOW DATABASES"))
databases = cursor.fetchall()
print(databases)

None
[('information_schema',), ('Sakila',), ('SharkSearch',), ('customer_db',), ('mysql',), ('performance_schema',), ('restaurantdata',), ('sys',)]


# Cuisine Table

In [3]:
# Create cuisine category
cursor.execute("DROP TABLE IF EXISTS cuisines")
cursor.execute("CREATE TABLE IF NOT EXISTS cuisines (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, cuisine_id INT(11), cuisine_category VARCHAR(255),UNIQUE KEY `cuisine_id` (`cuisine_id`))ENGINE=InnoDB")
cursor.execute("DESC cuisines")
print(cursor.fetchall())

[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('cuisine_id', 'int(11)', 'YES', 'UNI', None, ''), ('cuisine_category', 'varchar(255)', 'YES', '', None, '')]


# Restaurant Table

In [4]:
# Create restaurant table
cursor.execute("DROP TABLE IF EXISTS restaurants")
cursor.execute("CREATE TABLE IF NOT EXISTS restaurants (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, res_id INT(20), `name` VARCHAR(255), cuisines VARCHAR(255), cuisine_category VARCHAR(255), UNIQUE KEY `restaurants` (`res_id`)) ENGINE=InnoDB")
cursor.execute("DESC restaurants")
print(cursor.fetchall())


[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('res_id', 'int(20)', 'YES', 'UNI', None, ''), ('name', 'varchar(255)', 'YES', '', None, ''), ('cuisines', 'varchar(255)', 'YES', '', None, ''), ('cuisine_category', 'varchar(255)', 'YES', '', None, '')]


# Location Table

In [5]:
# Create location table
cursor.execute("DROP TABLE IF EXISTS location")
cursor.execute("CREATE TABLE IF NOT EXISTS location (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, `name` VARCHAR(255), lat DECIMAL(6,3), lng DECIMAL (6,3)) ENGINE=InnoDB")
cursor.execute("DESC location")
print(cursor.fetchall())


[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(255)', 'YES', '', None, ''), ('lat', 'decimal(6,3)', 'YES', '', None, ''), ('lng', 'decimal(6,3)', 'YES', '', None, '')]


#  Price Table

In [6]:
# Create price table
cursor.execute("DROP TABLE IF EXISTS price")
cursor.execute("CREATE TABLE IF NOT EXISTS price (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, `name` VARCHAR(255), price_range INT(2), ave_cost DECIMAL(6,2)) ENGINE=InnoDB")
cursor.execute("DESC price")
print(cursor.fetchall())


[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(255)', 'YES', '', None, ''), ('price_range', 'int(2)', 'YES', '', None, ''), ('ave_cost', 'decimal(6,2)', 'YES', '', None, '')]


# Media Table

In [7]:
# Create media table
cursor.execute("DROP TABLE IF EXISTS media")
cursor.execute("CREATE TABLE IF NOT EXISTS media (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, `name` VARCHAR(255), menu VARCHAR(255), featured_image VARCHAR(255)) ENGINE=InnoDB")
cursor.execute("DESC media")
print(cursor.fetchall())


[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(255)', 'YES', '', None, ''), ('menu', 'varchar(255)', 'YES', '', None, ''), ('featured_image', 'varchar(255)', 'YES', '', None, '')]


# Review Table

In [8]:
# Create reviews table
cursor.execute("DROP TABLE IF EXISTS reviews")
cursor.execute("CREATE TABLE IF NOT EXISTS reviews (id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, `name` VARCHAR(255), vote DECIMAL(6,2), rating DECIMAL(6,2)) ENGINE=InnoDB")
cursor.execute("DESC reviews")
print(cursor.fetchall())


[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('name', 'varchar(255)', 'YES', '', None, ''), ('vote', 'decimal(6,2)', 'YES', '', None, ''), ('rating', 'decimal(6,2)', 'YES', '', None, '')]


In [9]:
## getting all the tables which are present in 'datacamp' database
cursor.execute("SHOW TABLES")

tables = cursor.fetchall() ## it returns list of tables present in the database

## showing all the tables one by one
for table in tables:
    print(table)

('cuisines',)
('location',)
('media',)
('price',)
('restaurants',)
('reviews',)


# Load Data into MySqyl

## Load Cuisine Table

In [10]:
cuisine_values = []
def c_listify(v):
    return v["cuisine_id"], v["cuisine_category"] 

for v in cuisines_dict:
    entry_tuple = c_listify(v)
    cuisine_values.append(entry_tuple) 

In [11]:
## defining the Query
query = "INSERT INTO cuisines (cuisine_id, cuisine_category) VALUES (%s, %s)"

## storing values in a variable
values = cuisine_values

## executing the query with values
cursor.executemany(query, values)

## to make final output we have to run the 'commit()' method of the database object
db.commit()
print(cursor.rowcount, "records inserted")

108 records inserted


## Load Restaurant Data

In [12]:
restaurant_values = []
def r_listify(v):
    return v["res_id"], v["name"], v["cuisines"], v["cuisine_category"]

for v in restaurant_dict:
    entry_tuple = r_listify(v)
    cuisine_values.append(entry_tuple) 

In [13]:
## defining the Query
query = "INSERT INTO restaurants (res_id, name, cuisines, cuisine_category) VALUES (%s, %s, %s, %s)"

## storing values in a variable
values = restaurant_values

## executing the query with values
cursor.executemany(query, values)

## to make final output we have to run the 'commit()' method of the database object
db.commit()
print(cursor.rowcount, "records inserted")

108 records inserted


## Load Location Data

In [14]:
location_values = []
def l_listify(v):
    return v["name"], v["lat"], v["lng"]

for v in restaurant_dict:
    entry_tuple = l_listify(v)
    location_values.append(entry_tuple) 

In [15]:
## defining the Query
query = "INSERT INTO location (name, lat, lng) VALUES (%s, %s, %s)"

## storing values in a variable
values = location_values

## executing the query with values
cursor.executemany(query, values)

## to make final output we have to run the 'commit()' method of the database object
db.commit()
print(cursor.rowcount, "records inserted")

20 records inserted


## Load Price Data

In [16]:
price_values = []
def p_listify(v):
    return v["name"], v["price_range"], v["ave_cost"]

for v in restaurant_dict:
    entry_tuple = p_listify(v)
    price_values.append(entry_tuple) 

In [17]:
## defining the Query
query = "INSERT INTO price (name, price_range, ave_cost) VALUES (%s, %s, %s)"

## storing values in a variable
values = price_values

## executing the query with values
cursor.executemany(query, values)

## to make final output we have to run the 'commit()' method of the database object
db.commit()
print(cursor.rowcount, "records inserted")

20 records inserted


## Load Media Data

In [18]:
media_values = []
def m_listify(v):
    return v["name"], v["menu"], v["featured_image"]

for v in restaurant_dict:
    entry_tuple = m_listify(v)
    media_values.append(entry_tuple)

In [19]:
## defining the Query
query = "INSERT INTO media (name, menu, featured_image) VALUES (%s, %s, %s)"

## storing values in a variable
values = media_values

## executing the query with values
cursor.executemany(query, values)

## to make final output we have to run the 'commit()' method of the database object
db.commit()
print(cursor.rowcount, "records inserted")

20 records inserted


## Load Review Data

In [20]:
reviews_values = []
def r_listify(v):
    return v["name"], v["vote"], v["rating"]

for v in restaurant_dict:
    entry_tuple = r_listify(v)
    reviews_values.append(entry_tuple)

In [21]:
## defining the Query
query = "INSERT INTO reviews (name, vote, rating) VALUES (%s, %s, %s)"

## storing values in a variable
values = reviews_values

## executing the query with values
cursor.executemany(query, values)

## to make final output we have to run the 'commit()' method of the database object
db.commit()
print(cursor.rowcount, "records inserted")

20 records inserted


In [29]:
## defining the Query
query = "SELECT name FROM location"

## getting records from the table
cursor.execute(query)

## fetching all records from the 'cursor' object
records = cursor.fetchall()

## Showing the data
for record in records:
    print(record)

('Philippe, The Original',)
('Bottega Louie',)
('Pizzeria Mozza',)
('The Griddle Cafe',)
('Diddy Riese Cookies',)
('Wurstküche',)
('Gjelina',)
("Roscoe's House of Chicken & Waffles",)
('The Bazaar by Jose Andres',)
('Animal',)
('Bossa Nova Brazilian Cuisine',)
('Osteria Mozza',)
('The Ivy',)
('Canters',)
("Father's Office",)
('Providence',)
('In-N-Out Burger',)
('Alcove Cafe & Bakery',)
('The Apple Pan',)
('Daikokuya',)
